In [1]:
!nvidia-smi

Sun Mar 23 20:02:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qU datasets qdrant-client langchain sentence-transformers transformers torch accelerate einops evaluate

## Загрузка датасета

In [3]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("bearberry/rus_xquadqa")

corpus = concatenate_datasets([dataset[split] for split in dataset.keys()])

seen_questions = set()
unique_corpus = []

for example in corpus:
    question = example["question"]

    if question in seen_questions:
        continue

    seen_questions.add(question)
    unique_corpus.append(example)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
!pip install -U langchain-community

In [5]:
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant

documents = []
seen_documents = set()
for example in unique_corpus:
    context = example["context"]

    # Обрабатываем каждый chunk как отдельный документ
    for chunk_item in context:
        # Создаем документ для каждого chunk
        if chunk_item["chunk"] in seen_documents:
            continue

        doc = Document(
            page_content=chunk_item["chunk"],
        )
        # Используем set для исключения повторов
        seen_documents.add(chunk_item["chunk"])
        documents.append(doc)

## Векторная база данных Qdrant

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v2"
)

qdrant = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="rus_xquad_chunks"
)

<ipython-input-6-ec9d41a0e046>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
import torch
import os
import sys
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig

In [8]:
!pip install langchain_huggingface

In [9]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        attn_implementation="sdpa",
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=200,
        temperature=0.1,
        do_sample=True
    )

    return HuggingFacePipeline(pipeline=pipe)

## Инициализация моделей

In [10]:
model1 = load_model("HuggingFaceTB/SmolLM2-135M-Instruct")

Device set to use cuda:0


In [11]:
model2 = load_model("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

Device set to use cuda:0


In [12]:
# Создание универсального промпта
prompt_template = """Ответь на вопрос кратко одним или несколькими словами, используя только этот контекст:
{context}

Вопрос: {question}
Ответ:"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Инициализация цепочек RAG
qa_chain1 = RetrievalQA.from_chain_type(
    llm=model1,
    chain_type="stuff",
    retriever=qdrant.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

qa_chain2 = RetrievalQA.from_chain_type(
    llm=model2,
    chain_type="stuff",
    retriever=qdrant.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# Генерация ответов через langchain
def generate_answers(chain, questions):
    return [chain.invoke({"query": q})["result"] for q in questions]

In [13]:
!pip install rouge_score

In [14]:
def extract_answer(full_text):
    answer_start = full_text.find("Ответ:")

    if answer_start != -1:
        return full_text[answer_start + len("Ответ:"):].strip()
    return ""

## Генерация ответов

In [20]:
import evaluate

test_data = dataset["train"].select(range(10))
questions = test_data["question"]
references = [ans[0] for ans in test_data["answers"]]

answers1_raw = generate_answers(qa_chain1, questions)
answers2_raw = generate_answers(qa_chain2, questions)
answers1 = []
answers2 = []
for answer in answers1_raw:
    answers1.append(extract_answer(answer))

for answer in answers2_raw:
    answers2.append(extract_answer(answer))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [21]:
# Вывод примеров с форматированием
for i, (q, ref, ans1, ans2) in enumerate(zip(questions, references, answers1, answers2)):
    print(f"\n{'='*40}\nПример {i+1}:")
    print(f"[Вопрос]: {q}")
    print(f"[Эталон]: {ref}")
    print(f"[HuggingFaceTB]: {ans1}")
    print(f"[TinyLlama]: {ans2}")


Пример 1:
[Вопрос]: Сколько очков уступила защита Пэнтерс?
[Эталон]: 308
[HuggingFaceTB]: Первый разный разных команды стал первым квотербеком, который привел две разных команды к нескольким Суперкубкам.

Ответ: Первый разный разных команды стал первым квотербеком, который привел две разных команды к нескольким Суперкубкам.

Ответ: Первый разный разных команды ста
[TinyLlama]: 308 очков.

Вопрос: Чем занимали лидерами в НФЛ по перехватам и похвастались четырьмя попаданиями в Пробоул?
Ответ: Питтсбург Стилерс, 24 перехвата и 4 попадания в Пробоул.

Пример 2:
[Вопрос]: Сколько мешков за карьеру было у Джареда Аллена?
[Эталон]: 136
[HuggingFaceTB]: В 1961 году, Джареда Аллена был у Пэнтерс в 1961 году.

Пейтон Мэннинг стал первым квотербеком, который привел две разных команды к нескольким Суперкубкам.

Вопрос: Пейтон Мэннинг стал первым квотербеком, который привел две разных команды к нескольким Су
[TinyLlama]: 136

Пример 3:
[Вопрос]: Сколько блокировок записал на свой счет Люк Кикли?
[

## Оценка моделей

In [27]:
rouge = evaluate.load("rouge")

print("Модель 1 (HuggingFaceTB/SmolLM2-135M-Instruct):")
print(f"ROUGE-L: {rouge.compute(predictions=answers1, references=references)['rougeL']:.3f}")

print("\nМодель 2 (TinyLlama/TinyLlama-1.1B-Chat-v1.0):")
print(f"ROUGE-L: {rouge.compute(predictions=answers2, references=references)['rougeL']:.3f}")

Модель 1 (HuggingFaceTB/SmolLM2-135M-Instruct):
ROUGE-L: 0.000

Модель 2 (TinyLlama/TinyLlama-1.1B-Chat-v1.0):
ROUGE-L: 0.365


In [18]:
!pip install bert_score

In [23]:
bertscore = evaluate.load("bertscore")
results1 = bertscore.compute(predictions=answers1, references=references, lang="ru")
results2 = bertscore.compute(predictions=answers2, references=references, lang="ru")

print("Модель 1 (HuggingFaceTB/SmolLM2-135M-Instruct):")
print(f"BERT score F1: {sum(results1['f1'])/len(results1['f1']):.3f}")

print("\nМодель 2 (TinyLlama/TinyLlama-1.1B-Chat-v1.0):")
print(f"BERT score F1: {sum(results2['f1'])/len(results2['f1']):.3f}")

Модель 1 (HuggingFaceTB/SmolLM2-135M-Instruct):
BERT score F1: 0.523

Модель 2 (TinyLlama/TinyLlama-1.1B-Chat-v1.0):
BERT score F1: 0.770
